### Memory Tool 

https://docs.claude.com/en/docs/agents-and-tools/tool-use/memory-tool

In [1]:
# Step 1: Initialize the Anthropic client
import os
import anthropic
from pathlib import Path

# Initialize the Anthropic client
import anthropic
client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

In [2]:
# Step 2: Create the /scratchpad directory
# This directory will store Claude's persistent files (memory and bash working directory)

SCRATCHPAD_PATH = Path("./scratchpad")

# Create the directory if it doesn't exist
SCRATCHPAD_PATH.mkdir(exist_ok=True)

print(f"✓ Scratchpad directory created at: {SCRATCHPAD_PATH.absolute()}")
print(f"✓ Current contents: {list(SCRATCHPAD_PATH.iterdir())}")

✓ Scratchpad directory created at: /Users/rlm/Desktop/Code/open_claude_code/src/open_claude_agent/scratchpad
✓ Current contents: []


In [3]:
# Step 3: Import the memory tool handler from the dedicated module
# The handler has been moved to memory_tool.py for better code organization
from memory_tool import MemoryToolHandler

# Create a handler instance that will manage all memory operations
memory_handler = MemoryToolHandler(base_path="./scratchpad")

print("✓ Memory tool handler imported from memory_tool.py")

✓ Memory tool handler imported from memory_tool.py


In [4]:
# Step 4: Create a helper function to interact with Claude using the memory tool
# Import the reusable harness and configure it for the memory tool

from claude_harness import ClaudeHarness

# Create a harness configured for the memory tool
memory_harness = ClaudeHarness(
    client=client,
    model="claude-sonnet-4-5",
    max_tokens=4096,
    tools=[
        {
            "type": "memory_20250818",
            "name": "memory"
        },
    ],
    betas=["context-management-2025-06-27"],
    context_management={
        "edits": [
            {
                "type": "clear_tool_uses_20250919",
                "trigger": {
                    "type": "input_tokens",
                    "value": 100000
                },
                "keep": {
                    "type": "tool_uses",
                    "value": 3
                }
            }
        ]
    },
    tool_handlers={
        "memory": memory_handler.handle
    }
)

def call_claude_with_memory(user_message: str, max_turns: int = 5):
    """
    Send a message to Claude with the memory tool enabled.
    
    Args:
        user_message: The message to send to Claude
        max_turns: Maximum number of tool use iterations to prevent infinite loops
    """
    memory_harness.call(user_message, max_turns)

print("✓ Claude interaction helper created using reusable harness")

✓ Claude interaction helper created using reusable harness


### Test Examples

Now let's run several test examples to demonstrate the memory tool in action.

In [5]:
# Test Example 1: Ask Claude to store information in memory

call_claude_with_memory(
    "Please remember that my favorite color is blue and I prefer dark mode. "
    "Store this in your memory for future reference."
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: view                                                                                              │
│    Path: /memories                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Directory listing for /memories                                                                      │
│    Contents: []                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: create                                                                                            │
│    Path: /memories/user_preferences.txt                                                                         │
│    Params: {                                                                                                    │
│       "file_text": "User Preferences\n=================\n\nColor Preferences:\n- Favorite color:                │
│ blue\n\nDisplay Settings:\n- Preferred mode: dark mode\n"                                                       │
│ }                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Created file: /memories/user_preferences.txt                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 💬 Claude ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Perfect! I've stored your preferences in my memory:                                                            │
│  - Favorite color: **blue**                                                                                     │
│  - Display preference: **dark mode**                                                                            │
│                                                                                                                 │
│  I'll remember these preferences for our future interactions.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
# Test Example 2: Ask Claude to recall stored information
call_claude_with_memory(
    "What are my preferences that you have stored?"
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: view                                                                                              │
│    Path: /memories                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Directory listing for /memories                                                                      │
│    Contents: ['user_preferences.txt']                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: view                                                                                              │
│    Path: /memories/user_preferences.txt                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: File contents: /memories/user_preferences.txt                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 💬 Claude ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Currently, I have a preferences file created but it's empty - I don't have any preferences stored for you      │
│  yet. Would you like to tell me about your preferences so I can save them for future reference? This could      │
│  include things like:                                                                                           │
│                                                                                                                 │
│  - Communication style preferences                                                                              │
│  - Topics of interest                                                                                           │
│  - How you like information presented                                                                           │
│  - Any specific needs or requirements                                                                           │
│  - Or anything else you'd like me to remember about you                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
# Test Example 3: Ask Claude to update stored information
call_claude_with_memory(
    "Actually, I've changed my mind. My favorite color is now green. "
    "Please update your memory."
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: view                                                                                              │
│    Path: /memories                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Directory listing for /memories                                                                      │
│    Contents: ['user_preferences.txt']                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: view                                                                                              │
│    Path: /memories/user_preferences.txt                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: File contents: /memories/user_preferences.txt                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: create                                                                                            │
│    Path: /memories/user_preferences.txt                                                                         │
│    Params: {                                                                                                    │
│       "file_text": "User Preferences\n=================\n\nFavorite color: green\n"                             │
│ }                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Created file: /memories/user_preferences.txt                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 💬 Claude ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Done! I've updated my memory to record that your favorite color is now green. 🟢                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Bash Tool 

https://docs.claude.com/en/docs/agents-and-tools/tool-use/bash-tool

Now let's demonstrate the bash tool using the same reusable harness!

In [7]:
# Step 1: Import the bash tool handler
from bash_tool import BashToolHandler

# Create a bash tool handler instance that works in the scratchpad directory
bash_handler = BashToolHandler(timeout=60, working_dir="./scratchpad")

print("✓ Bash tool handler imported from bash_tool.py")
print(f"✓ Bash working directory: {SCRATCHPAD_PATH.absolute()}")

✓ Bash tool handler imported from bash_tool.py
✓ Bash working directory: /Users/rlm/Desktop/Code/open_claude_code/src/open_claude_agent/scratchpad


In [8]:
# Step 2: Configure Claude harness for the bash tool

bash_harness = ClaudeHarness(
    client=client,
    model="claude-sonnet-4-5",
    max_tokens=4096,
    tools=[
        {
            "type": "bash_20250124",
            "name": "bash"
        },
    ],
    betas=["computer-use-2025-01-24"],
    tool_handlers={
        "bash": bash_handler.handle
    }
)

def call_claude_with_bash(user_message: str, max_turns: int = 5):
    """
    Send a message to Claude with the bash tool enabled.
    
    Args:
        user_message: The message to send to Claude
        max_turns: Maximum number of tool use iterations to prevent infinite loops
    """
    bash_harness.call(user_message, max_turns)

print("✓ Claude interaction helper created for bash tool using reusable harness")

✓ Claude interaction helper created for bash tool using reusable harness


In [9]:
# Test Example: Ask Claude to use bash commands

call_claude_with_bash(
    "Can you check what files are in the current directory using ls? "
    "Then create a test file called hello.txt with some content."
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: bash                                                                                              │
│    Command: ls -la                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Command executed successfully                                                                        │
│    Output: total 0                                                                                              │
│ drwxr-xr-x@  3 rlm  staff   96 Oct 24 08:18 .                                                           │
│ drwxr-xr-x@ 10 rlm  staff  320 Oct 24 08:17 ..                                                          │
│ -rw-r--r--@  1 rlm  staff    0 Oct 24 08:18 user_preferences.txt                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: bash                                                                                              │
│    Command: echo "Hello! This is a test file with some sample content." > hello.txt                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Command executed successfully                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: bash                                                                                              │
│    Command: ls -la && echo "--- Content of hello.txt ---" && cat hello.txt                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✓ Success: Command executed successfully                                                                        │
│    Output: total 8                                                                                              │
│ drwxr-xr-x@  4 rlm  staff  128 Oct 24 08:19 .                                                           │
│ drwxr-xr-x@ 10 rlm  staff  320 Oct 24 08:17 ..                                                          │
│ -rw-r--r--@  1 rlm  staff   53 Oct 24 08:19 hello.txt                                                           │
│ -rw-r--r--@  1 rlm  sta...                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 💬 Claude ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Perfect! Here's what I found:                                                                                  │
│                                                                                                                 │
│  **Current directory contents:**                                                                                │
│  - `user_preferences.txt` - an empty file that was already there                                                │
│  - `hello.txt` - the new file I just created (53 bytes)                                                         │
│                                                                                                                 │
│  **Content of hello.txt:**                                                                                      │
│  ```                                                                                                            │
│  Hello! This is a test file with some sample content.                                                           │
│  ```                                                                                                            │
│                                                                                                                 │
│  The file has been successfully created!                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Summary

This notebook demonstrates a minimal implementation of Claude's memory and bash tools using a reusable harness:

1. **SDK Setup**: We initialized the Anthropic client with a reusable `ClaudeHarness` that can be configured for different tools

2. **Scratchpad Directory**: Created a `./scratchpad` directory that serves as:
   - Storage for Claude's persistent memory files (memory tool)
   - Working directory for bash command execution (bash tool)

3. **Memory Tool Handler**: Implemented in `memory_tool.py`, processes 6 memory commands:
   - `view`: List directories or read files
   - `create`: Create/overwrite files
   - `str_replace`: Replace text in files
   - `insert`: Insert text at specific lines
   - `delete`: Remove files/directories
   - `rename`: Move or rename files

4. **Bash Tool Handler**: Implemented in `bash_tool.py`, executes bash commands in the scratchpad directory with timeout protection

5. **Test Examples**: Demonstrated both tools with several interactions showing Claude:
   - Storing and retrieving information in memory files
   - Executing bash commands and viewing outputs
   - All operations are isolated in the scratchpad directory

**Key Takeaways**:
- Both tools are client-side: your application executes all operations locally
- Security is critical: paths are validated and commands run in a sandboxed directory
- The reusable harness makes it easy to add new tools with minimal code
- Claude automatically checks the scratchpad before starting tasks when tools are enabled